In [1]:
# Check PyTorch version installed on this system
!python -c "import torch; print(torch.__version__)"

1.13.0+cu116


In [2]:
!nvidia-smi

Thu Dec 15 02:21:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Download the required modules
"""
Assign to TORCH with what you get from the cell above, E.g., export TORCH=1.13.0+cu116
"""
%env TORCH=1.13.0+cu116
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-geometric
!pip install torchmetrics
!pip install matplotlib
!pip install networkx
!pip install scipy

env: TORCH=1.13.0+cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
     |████████████████████████████████| 9.4 MB 14.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
     |████████████████████████████████| 4.6 MB 14.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 564 kB 15.0 MB/s 
     |████████████████████████████████| 280 kB 71.6 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=5d664f5cdaae8241454687e47e7f10062c633701cf4bc5bffb87cc88bcc055be
  Stored in directory: /root/.cache/pip/wheels/59/a3/20/198928106d3169865ae73afcbd3d3d1796cf6b429b55c65378
Successfully built torch-geometric
  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 512 kB 14.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
github_username=""
github_repository="grl"
github_token = ""

%env github_repository={github_repository}

env: github_repository=grl


In [5]:
!git clone https://{github_token}@github.com/{github_username}/{github_repository}.git

Cloning into 'grl'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 34 (delta 6), reused 29 (delta 4), pack-reused 0
Unpacking objects: 100% (34/34), done.


In [6]:
%cd $github_repository

/content/grl


In [7]:
import torch
import random

random.seed(0)
torch.manual_seed(0)

In [8]:
from torch_geometric.datasets import PPI
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import AddSelfLoops
import dataset

train_dataset = PPI(root='', split='train', transform=AddSelfLoops())
val_dataset = PPI(root='', split='val', transform=AddSelfLoops())
test_dataset = PPI(root='', split='test', transform=AddSelfLoops())

dataset.num_features = 50
dataset.num_labels = 121
dataset.train_loader = DataLoader(train_dataset, batch_size=2)
dataset.val_loader = DataLoader(val_dataset, batch_size=2)
dataset.test_loader = DataLoader(test_dataset, batch_size=2)

Extracting ./ppi.zip
Processing...
Done!


In [9]:
from evaluation import evaluate
from training_loop import train

In [10]:
from models import GATv1PPI
model = GATv1PPI(dataset.num_features, dataset.num_labels)
evaluate(model, dataset.test_loader)

(tensor(0.7062, device='cuda:0'), tensor(0.3849, device='cuda:0'))

In [11]:
ppi_train_params = {
  "lr": 5e-3,
  "weight_decay": 0,
  "epochs": 400,
  "patience": 100,
  "model_name": model.model_name
}

In [12]:
def run_experiment(iters=10):
  losses = []
  scores = []
  for iter in range(iters):
    best_model, _, _, _, _, _ = train(model, ppi_train_params, verbose=False)
    loss, score = evaluate(best_model, dataset.test_loader)
    losses.append(loss)
    scores.append(score)
  losses = torch.tensor(losses)
  scores = torch.tensor(scores)
  return (torch.std_mean(losses), torch.std_mean(scores))

In [13]:
loss_ci, score_ci = run_experiment(10)

training model GATv1PPI
GATv1PPI(
  (layers): ModuleList(
    (0): GATLayer()
    (1): GATLayer()
    (2): GATLayer(
      (act): Identity()
    )
  )
)
training...
best model performance @ epoch 00176: 
	val_loss: 0.1008 | val_micro_f1: 0.9593
	test_loss: 0.0621 | test_micro_f1: 0.9750
training model GATv1PPI
GATv1PPI(
  (layers): ModuleList(
    (0): GATLayer()
    (1): GATLayer()
    (2): GATLayer(
      (act): Identity()
    )
  )
)
training...
best model performance @ epoch 00118: 
	val_loss: 0.0996 | val_micro_f1: 0.9553
	test_loss: 0.0611 | test_micro_f1: 0.9728
training model GATv1PPI
GATv1PPI(
  (layers): ModuleList(
    (0): GATLayer()
    (1): GATLayer()
    (2): GATLayer(
      (act): Identity()
    )
  )
)
training...
best model performance @ epoch 00180: 
	val_loss: 0.0943 | val_micro_f1: 0.9628
	test_loss: 0.0603 | test_micro_f1: 0.9754
training model GATv1PPI
GATv1PPI(
  (layers): ModuleList(
    (0): GATLayer()
    (1): GATLayer()
    (2): GATLayer(
      (act): Identi

In [14]:
loss_std, loss_mean = loss_ci
score_std, score_mean = score_ci

print(f'loss:\t\t{loss_mean:.4f} +/- {loss_std:.4f}')
print(f'micro F1 score: {score_mean:.4f} +/- {score_std:.4f}')

loss:		0.0595 +/- 0.0024
micro F1 score: 0.9766 +/- 0.0019
